In [1]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 8
MAX_LIST_LEN = 8
MAX_STEPS = 210

SUCCESS_REWARD = 2.0
STEP_REWARD = -0.1
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.8
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma08_step210_v1'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [3]:
# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    # Optionally, load a checkpoint
    # load_checkpoint("path_to_checkpoint", model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    total_steps = 0
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if done and reward == SUCCESS_REWARD:
                        success = True
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if done and reward == SUCCESS_REWARD:
                    success = True
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        if success: 
            num_successes += 1
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            success_rate = num_successes / episode_cnt
            avg_steps = total_steps / episode_cnt
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns = []
            R = 0
            gamma = GAMMA
            for r in rewards[::-1]:
                R = r + gamma * R
                returns.insert(0, R)
            returns = torch.tensor(returns).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        total_steps += len(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{len(rewards)}, total reward:{sum(rewards):.4f}, {t2-t1} sec")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss:-47.1221, fail, steps:8, total reward:-11.2000, 0.3784205913543701 sec
Episode 1, loss:-43.4430, fail, steps:11, total reward:-9.3000, 0.026682376861572266 sec
Episode 2, loss:-5.9498, fail, steps:1, total reward:-10.0000, 0.0031452178955078125 sec
Episode 3, loss:-3.2463, fail, steps:1, total reward:-10.0000, 0.0028829574584960938 sec
Episode 4, loss:-58.3752, fail, steps:46, total reward:-12.4000, 0.10956645011901855 sec
Episode 5, loss:-48.6245, fail, steps:14, total reward:-9.4000, 0.030432939529418945 sec
Episode 6, loss:-7.0790, fail, steps:1, total reward:-10.0000, 0.003426074981689453 sec
Episode 7, loss:-9.6680, fail, steps:1, total reward:-10.0000, 0.002936124801635742 sec
Episode 8, loss:-59.8424, fail, steps:108, total reward:-11.3000, 0.32430577278137207 sec
Episode 9, loss:-91.3233, fail, steps:18, total reward:-10.5000, 0.04279661178588867 sec
Episode 10, loss:-75.9857, fail, steps:137, total reward:-14.4000, 0.300731897354126 sec
Episode 11, loss:-13.047

Episode 92, loss:-1.4131, fail, steps:160, total reward:3.1000, 0.3614506721496582 sec
Episode 93, loss:-32.2701, fail, steps:68, total reward:-5.0000, 0.16035079956054688 sec
Episode 94, loss:3.8853, fail, steps:158, total reward:-0.1000, 0.3439908027648926 sec
Episode 95, loss:1.9031, fail, steps:159, total reward:1.0000, 0.35780858993530273 sec
Episode 96, loss:14.5926, fail, steps:160, total reward:5.1000, 0.3662145137786865 sec
Episode 97, loss:-9.1198, fail, steps:159, total reward:-0.4000, 0.36095690727233887 sec
Episode 98, loss:4.9960, fail, steps:160, total reward:3.1000, 0.3644695281982422 sec
Episode 99, loss:14.1326, fail, steps:160, total reward:5.1000, 0.3646979331970215 sec
Episode 100, loss:-42.0095, fail, steps:12, total reward:-8.3000, 0.039658308029174805 sec
Episode 101, loss:-6.5579, fail, steps:159, total reward:1.0000, 0.33654212951660156 sec
Episode 102, loss:3.0482, fail, steps:159, total reward:3.0000, 0.35297703742980957 sec
Episode 103, loss:28.0112, fail, 

Episode 185, loss:6.1520, fail, steps:161, total reward:5.1000, 0.3470301628112793 sec
Episode 186, loss:21.1964, fail, steps:160, total reward:8.1000, 0.3567042350769043 sec
Episode 187, loss:28.2703, fail, steps:161, total reward:8.2000, 0.3638033866882324 sec
Episode 188, loss:25.4871, succeed, steps:98, total reward:9.0000, 0.22624754905700684 sec
Episode 189, loss:-1.6484, fail, steps:159, total reward:1.8000, 0.3533635139465332 sec
Episode 190, loss:4.7634, fail, steps:160, total reward:3.3000, 0.3625655174255371 sec
Episode 191, loss:11.6588, fail, steps:159, total reward:4.0000, 0.36226654052734375 sec
Episode 192, loss:34.4433, succeed, steps:89, total reward:9.4000, 0.20630335807800293 sec
Episode 193, loss:3.6548, succeed, steps:123, total reward:2.7000, 0.27242517471313477 sec
Episode 194, loss:-35.7212, fail, steps:53, total reward:-2.6000, 0.1222066879272461 sec
Episode 195, loss:-91.0948, fail, steps:37, total reward:-9.0000, 0.08188509941101074 sec
Episode 196, loss:31.

Episode 277, loss:-52.6767, fail, steps:152, total reward:-7.8000, 0.34328532218933105 sec
Episode 278, loss:18.8166, fail, steps:161, total reward:8.2000, 0.3677396774291992 sec
Episode 279, loss:19.0662, succeed, steps:86, total reward:6.4000, 0.1991269588470459 sec
Episode 280, loss:-6.9821, fail, steps:160, total reward:1.0000, 0.35135746002197266 sec
Episode 281, loss:15.6804, fail, steps:160, total reward:6.1000, 0.3603394031524658 sec
Episode 282, loss:18.0852, fail, steps:160, total reward:6.1000, 0.3626575469970703 sec
Episode 283, loss:17.3528, succeed, steps:78, total reward:4.6000, 0.18213653564453125 sec
Episode 284, loss:-44.6498, fail, steps:46, total reward:-5.3000, 0.10414361953735352 sec
Episode 285, loss:16.7439, succeed, steps:101, total reward:5.7000, 0.21490168571472168 sec
Episode 286, loss:8.1719, succeed, steps:136, total reward:3.7000, 0.30690455436706543 sec
Episode 287, loss:2.1538, fail, steps:160, total reward:4.1000, 0.3542780876159668 sec
Episode 288, lo

...

Episode 12001, loss:1.4107, succeed, steps:155, total reward:12.7000, 0.6811857223510742 sec
Episode 12002, loss:6.8436, succeed, steps:101, total reward:15.2000, 0.46207618713378906 sec
Episode 12003, loss:4.8372, succeed, steps:99, total reward:16.3000, 0.39141273498535156 sec
Episode 12004, loss:3.2569, succeed, steps:53, total reward:7.5000, 0.21210169792175293 sec
Episode 12005, loss:2.1936, succeed, steps:75, total reward:10.9000, 0.37758755683898926 sec
Episode 12006, loss:1.9911, succeed, steps:115, total reward:10.6000, 0.4508171081542969 sec
Episode 12007, loss:0.3330, succeed, steps:100, total reward:14.2000, 0.45482683181762695 sec
Episode 12008, loss:1.9535, succeed, steps:92, total reward:12.4000, 0.3642451763153076 sec
Episode 12009, loss:1.2116, fail, steps:161, total reward:9.1000, 0.6877660751342773 sec
Episode 12010, loss:0.0857, succeed, steps:92, total reward:9.1000, 0.37610578536987305 sec
Episode 12011, loss:0.8355, succeed, steps:96, total reward:16.4000, 0.4934

Episode 12090, loss:1.1119, succeed, steps:89, total reward:12.5000, 0.4435560703277588 sec
Episode 12091, loss:4.8196, succeed, steps:87, total reward:13.6000, 0.3415858745574951 sec
Episode 12092, loss:0.5469, succeed, steps:87, total reward:13.6000, 0.3508486747741699 sec
Episode 12093, loss:1.2305, succeed, steps:78, total reward:7.7000, 0.41962122917175293 sec
Episode 12094, loss:2.0526, succeed, steps:78, total reward:7.7000, 0.2953474521636963 sec
Episode 12095, loss:1.9776, succeed, steps:66, total reward:11.2000, 0.27817583084106445 sec
Episode 12096, loss:0.4304, succeed, steps:109, total reward:7.7000, 0.4616219997406006 sec
Episode 12097, loss:3.2575, succeed, steps:120, total reward:15.6000, 0.479170560836792 sec
Episode 12098, loss:0.9606, succeed, steps:126, total reward:9.2000, 0.5866439342498779 sec
Episode 12099, loss:0.4854, succeed, steps:152, total reward:19.7000, 0.677056074142456 sec
Episode 12100, loss:3.5917, succeed, steps:61, total reward:9.3000, 0.2443630695

Episode 12179, loss:3.1116, fail, steps:162, total reward:13.1000, 0.6432664394378662 sec
Episode 12180, loss:1.4669, succeed, steps:82, total reward:8.6000, 0.4277212619781494 sec
Episode 12181, loss:2.0414, succeed, steps:101, total reward:12.1000, 0.3985426425933838 sec
Episode 12182, loss:3.8804, succeed, steps:74, total reward:9.8000, 0.29721736907958984 sec
Episode 12183, loss:1.5310, succeed, steps:23, total reward:2.2000, 0.19266939163208008 sec
Episode 12184, loss:2.2650, succeed, steps:76, total reward:11.9000, 0.3164982795715332 sec
Episode 12185, loss:0.8448, succeed, steps:67, total reward:9.1000, 0.262620210647583 sec
Episode 12186, loss:2.0930, succeed, steps:111, total reward:12.8000, 0.42380189895629883 sec
Episode 12187, loss:3.4240, succeed, steps:92, total reward:12.4000, 0.4456326961517334 sec
Episode 12188, loss:1.4730, succeed, steps:105, total reward:13.0000, 0.42969846725463867 sec
Episode 12189, loss:3.0508, succeed, steps:106, total reward:13.9000, 0.48859930

Episode 12269, loss:3.4885, succeed, steps:129, total reward:15.3000, 0.5639941692352295 sec
Episode 12270, loss:-0.5365, fail, steps:161, total reward:10.2000, 0.6867258548736572 sec
Episode 12271, loss:1.9465, succeed, steps:113, total reward:11.7000, 0.45587873458862305 sec
Episode 12272, loss:1.1049, succeed, steps:119, total reward:14.6000, 0.5470137596130371 sec
Episode 12273, loss:-0.7306, fail, steps:160, total reward:5.0000, 0.6725361347198486 sec
Episode 12274, loss:1.4732, succeed, steps:131, total reward:11.1000, 0.5375545024871826 sec
Episode 12275, loss:0.9162, succeed, steps:88, total reward:11.5000, 0.39467477798461914 sec
Episode 12276, loss:1.6827, succeed, steps:111, total reward:12.8000, 0.4443366527557373 sec
Episode 12277, loss:0.1196, succeed, steps:51, total reward:8.6000, 0.312546968460083 sec
Episode 12278, loss:3.5781, succeed, steps:141, total reward:14.9000, 0.5515551567077637 sec
Episode 12279, loss:5.8418, fail, steps:163, total reward:17.3000, 0.73977017

KeyboardInterrupt: 